This notebook is a basic tutorial that demonstrates how to configure a simulation using Concordia.

<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/selling_cookies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Colab-specific setup (use a CodeSpace to avoid the need for this).
try:
  %env COLAB_RELEASE_TAG
except:
  pass  # Not running in colab.
else:
  %pip install --ignore-requires-python --requirement 'https://raw.githubusercontent.com/google-deepmind/concordia/main/examples/requirements.in' 'git+https://github.com/google-deepmind/concordia.git#egg=gdm-concordia'
  %pip list

In [ ]:
# @title Imports

import numpy as np
from IPython import display

import sentence_transformers

from concordia.language_model import utils as language_model_utils

from concordia.prefabs.simulation import generic as simulation

import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs

from concordia.typing import prefab as prefab_lib
from concordia.typing import scene as scene_lib
from collections.abc import Mapping, Sequence
from concordia.typing import entity as entity_lib

from concordia.utils import helper_functions

In [ ]:
# @title Language Model Selection: provide key or select DISABLE_LANGUAGE_MODEL

# By default this colab uses models via an external API so you must provide an
# API key. TogetherAI offers open weights models from all sources.

API_KEY = ''  #@param {type: 'string'}
# See concordia/language_model/utils.py
API_TYPE = 'openai'  # e.g. 'together_ai' or 'openai'.
MODEL_NAME = 'gpt-5'  # for API_TYPE = 'together_ai', we recommend MODEL_NAME = 'google/gemma-3-27b-it'
# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True
DISABLE_LANGUAGE_MODEL = False

In [ ]:
# @title Use the selected language model

# Note that it is also possible to use local models or other API models,
# simply replace this cell with the correct initialization for the model
# you want to use.

if not DISABLE_LANGUAGE_MODEL and not API_KEY:
  raise ValueError('API_KEY is required.')

model = language_model_utils.language_model_setup(
    api_type=API_TYPE,
    model_name=MODEL_NAME,
    api_key=API_KEY,
    disable_language_model=DISABLE_LANGUAGE_MODEL,
)


In [ ]:
# @title Setup sentence encoder

if DISABLE_LANGUAGE_MODEL:
  embedder = lambda _: np.ones(3)
else:
  st_model = sentence_transformers.SentenceTransformer(
      'sentence-transformers/all-mpnet-base-v2')
  embedder = lambda x: st_model.encode(x, show_progress_bar=False)

In [ ]:
test = model.sample_text(
    'Is societal and technological progress like getting a clearer picture of '
    'something true and deep?')
print(test)

In [ ]:
# @title Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
#@title Print menu of prefabs

display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))

# Two Friends Catching Up

In [ ]:
instances = [
    prefab_lib.InstanceConfig(
        prefab='conversational__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': 'Jessica',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='conversational__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': 'Chloe',
            'conversation_style': 'Speaks with a lot of energy and excitement. Uses casual, modern language and slang common for a Southern California girl in their mid-20s.'
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='dialogic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'conversation rules',
            "next_game_master_name": "conversation rules",
            "acting_order": "fixed",
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': 'initial setup rules',
            'next_game_master_name': 'conversation rules',
            # Shared memory establishes the immediate context for their meeting.
            'shared_memories': ["Jessica and Chloe are best friends from college meeting at their favorite coffee shop to catch up after not seeing each other for months."],
            # Player-specific memories give them individual things to talk about.
            'player_specific_memories': {
                'Jessica' : ['Recently got a big promotion at her marketing job and is thinking about moving in with her boyfriend.'],
                'Chloe' : ['Just got back from a whirlwind backpacking trip through Europe and has tons of stories to share.'],
            },
        },
    ),
]

In [ ]:
config = prefab_lib.Config(
    default_premise=f'Two people friends over coffee to discuss their lives and catch up',
    default_max_steps=20,
    prefabs=prefabs,
    instances=instances,
)

In [ ]:
# @title Initialize the simulation
raw_log = []
runnable_simulation = simulation.Simulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [ ]:
# @title Run the simulation
results_log = runnable_simulation.play(raw_log = raw_log)

In [ ]:
# @title Display the log
display.HTML(results_log)

# Love Island First Date

In [ ]:
PLAYER_ONE  = 'Cody'
PLAYER_TWO  = 'Megan'

instances = [
    prefab_lib.InstanceConfig(
        prefab='conversational__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_ONE,
            'conversation_style': 'Talk like a charismatic and flirty stud from Miami.'
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='conversational__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_TWO,
            'conversation_style': 'Talk like a stereotypical Love Island bombshell from Sussex.',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='dialogic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'conversation rules',
            "next_game_master_name": "conversation rules",
            "acting_order": "fixed",
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': 'initial setup rules',
            'next_game_master_name': 'conversation rules',
            'player_specific_memories': {
                PLAYER_ONE : [
                    'Is a confident personal trainer from Miami, USA.',
                    'He is on a first date in the Love Island villa.',
                    'He thinks English girls are "unreal".'
                ],
                PLAYER_TWO : [
                    "Is a classic 'bombshell' from Sussex.",
                    'She is on a first date in the Love Island villa.',
                    'She is not afraid to "step on toes" to find her man.',
                ],
            },
        },
    ),
]

config = prefab_lib.Config(
    default_premise=f'Night has fallen on the Love Island villa. The iconic fire pit is lit, casting a warm glow. {PLAYER_ONE} and {PLAYER_TWO} are sat on the curved sofa around the fire, drinks in hand. The other islanders are out of earshot, giving them their first chance to properly graft.',
    default_max_steps=60,
    prefabs=prefabs,
    instances=instances,
)

In [ ]:
# @title Initialize the simulation
raw_log = []
runnable_simulation = simulation.Simulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [ ]:
# @title Run the simulation
results_log = runnable_simulation.play(raw_log = raw_log)

In [ ]:
# @title Display the log
display.HTML(results_log)

```
Copyright 2025 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```